In [2]:
import util
%precision 4
%matplotlib inline


In [63]:
# Shotlink data files (not available in git - visit pga site for getting research access)
files = ['data/rshot-2014.txt', 'data/rshot-2015.txt', 'data/rshot-2016.txt']

In [ ]:
# this could be done using psql batch imports.  this works, is convenient but slow
for f in files:
    # inner function to convert column names from flat file header names
    def convert(c):
        return c.lower().replace(' ', '_').replace('/', '_').replace('.', '_').replace('#', 'id').replace('(', '_').replace(')', '').replace('id_of', 'num_of')
    df = pd.read_csv(filepath_or_buffer=f, sep=';', skipinitialspace=True)
    df.columns = [convert(col) for col in df.columns]
    df['date'] = pd.to_datetime(df['date'])
    df.to_sql(name= 'strokes', con= golf_engine, if_exists= 'append')

In [32]:
# aggregate scores for players, tournaments, years, rounds
sql_txt = '''
drop table if exists scores cascade;
create table scores (
     year        integer,
     date        timestamp,
     player_id   integer,
     permanent_tournament_id    integer,
     round       integer,
     score       integer,
     sg_tee      float,
     sg_approach float,
     sg_around   float,
     sg_putting  float,
     primary key (year, date, player_id, permanent_tournament_id, round) 
);

create view scores_view AS
  select * from scores
  order by year, player_id, permanent_tournament_id, date, round;
'''
exec_sql(sql_txt)

In [33]:
sql_txt = '''
    insert into scores
    select    year, date, player_id, permanent_tournament_id, round,  count(shot)
        from    strokes 
        group by year, date, player_id, permanent_tournament_id, round
        order by year, player_id, permanent_tournament_id, date, round;
'''
exec_sql(sql_txt)

In [37]:
# combine rounds
sql_txt = '''
    create or replace function combine_rounds()
        returns integer AS $$
    declare 
        num_recs integer = 0;
        first boolean = true;
        score_rec   record;
        prev_score_rec record;
        scores_cur cursor 
            for select *
            from scores_view;
    begin
        open scores_cur;
        loop
            fetch scores_cur into score_rec;
            exit when not found;
 
            if not first then 
                if  score_rec.round = prev_score_rec.round 
                    and score_rec.player_id = prev_score_rec.player_id
                    and score_rec.permanent_tournament_id = prev_score_rec.permanent_tournament_id
                    and score_rec.year = prev_score_rec.year
                then 
                    update scores 
                        set score = score + prev_score_rec.score 
                        where year = score_rec.year
                        and   player_id = score_rec.player_id
                        and   date = score_rec.date
                        and   permanent_tournament_id = score_rec.permanent_tournament_id
                        and   round = score_rec.round;
                    delete from scores 
                        where year = prev_score_rec.year
                        and   player_id = prev_score_rec.player_id
                        and   date = prev_score_rec.date
                        and   permanent_tournament_id = prev_score_rec.permanent_tournament_id
                        and   round = prev_score_rec.round;
                    num_recs := num_recs + 1;
                    score_rec.score = score_rec.score + prev_score_rec.score;
                end if;
            else
                first = false;
            end if;
            
            prev_score_rec = score_rec;
        end loop;
  
        close scores_cur;
 
        return num_recs;
    end; $$

    language plpgsql;
'''
exec_sql(sql_txt)
exec_sql('select combine_rounds()')

In [97]:
sql_txt = '''
    drop view if exists stats_view cascade;
    create view stats_view as 
        select s.year, s.player_id, name, avg(score) as avg_score, stddev(score) as std_score, 
             avg(sg_tee) as avg_sg_tee, avg(sg_approach) as avg_sg_approach, 
             avg(sg_around) as avg_sg_around, avg(sg_putting) as avg_sg_putting
          from scores as s, players p 
          where s.player_id = p.player_id 
          group by s.year, name, s.player_id
          having count(score) > 25
          order by year, avg_score;
'''
exec_sql(sql_txt)


In [98]:
sql_txt = '''
    drop view if exists rank_view;
    create view rank_view as
        select year, player_id, row_number() over (order by avg_score) from stats_view;
'''
exec_sql(sql_txt)

In [38]:
#  DELETE ROUNDS WITHOUT 18 holes
sql_txt = '''
    delete from scores s
        where exists (select year, player_id, permanent_tournament_id, round, count(hole) 
            from (select    distinct year, player_id, permanent_tournament_id, round,  hole
                  from    strokes) b
              where b.year = s.year 
                and b.player_id = s.player_id
                and b.permanent_tournament_id = s.permanent_tournament_id
                and b.round = s.round
                group by year, player_id, permanent_tournament_id, round
                having count(hole) < 18
    );
'''
exec_sql(sql_txt)


In [19]:
sql_txt = '''
    create or replace function update_strokes_gained(category text, category_field text)
        returns integer as $$
    declare 
        num_recs integer = 0;
        round_rec   record;
        round_cur cursor for
            select year, player_id, permanent_tournament_id, round, sum(strokes_gained_baseline) as sg
            from strokes
            where strokes_gained_category like category
            group by year, player_id, permanent_tournament_id, round;
    begin
        open round_cur;
 
        loop
            fetch round_cur INTO round_rec;
            exit when not found;

            execute 'update scores 
                set ' || category_field || ' = ' || round_rec.sg ||
                ' where year = ' || round_rec.year || '
                and player_id = ' || round_rec.player_id || '
                and permanent_tournament_id = ' || round_rec.permanent_tournament_id || '
                and round = ' || round_rec.round || ';';
            num_recs = num_recs + 1;
        end loop;

       close round_cur;
 
       return num_recs;
    end; $$
    language plpgsql;
'''
util.exec_sql(sql_txt)
util.exec_sql("select update_strokes_gained('Off the Tee', 'sg_tee')")
util.exec_sql("select update_strokes_gained('Approach to the Green', 'sg_approach')")
util.exec_sql("select update_strokes_gained('Around the Green', 'sg_around')")
util.exec_sql("select update_strokes_gained('Putting', 'sg_putting')")

In [14]:
sql_txt = '''
    drop type if exists stat cascade;
    create type stat as (
        mean float, 
        se float, 
        sd float
    );

    drop table if exists stan_tournaments cascade;
    create table stan_tournaments (
        permanent_tournament_id     integer,
        year          integer,
        tag           varchar(50),
        alpha         stat,
        primary key (permanent_tournament_id, year, tag) 
    );
'''
util.exec_sql(sql_txt)

In [4]:
sql_txt = '''
    drop table if exists stan_players cascade;
    create table stan_players (
        player_id     integer,
        year          integer,
        tag           varchar(50),
        tau           stat,
        sigma         stat,
        primary key (player_id, year, tag) 
    );
'''
util.exec_sql(sql_txt)

In [23]:
sql_txt = '''
    drop table if exists stan_sg_tournaments cascade;
    create table stan_sg_tournaments (
         permanent_tournament_id     integer,
         year                        integer,
         tag                     varchar(50),
         b_offset                      stat,
         b_tee                         stat,
         b_approach                    stat,
         b_around                      stat,
         b_putting                     stat,
         primary key (permanent_tournament_id, year, tag) 
    );
'''
util.exec_sql(sql_txt)

In [6]:
sql_txt = '''
    drop type if exists full_stat cascade;
    create type full_stat as (
        mean float, 
        se float, 
        sd float, 
        p_025 float, 
        p_25 float, 
        p_50 float, 
        p_75 float, 
        p_975 float
    );
                                                   
    drop table if exists stan_prediction;
    create table stan_prediction (
        player_id                   integer,
        permanent_tournament_id     integer,
        year                        integer,
        tag                         varchar(50),
        round                       integer,
        score                       full_stat,
        primary key (player_id, permanent_tournament_id, year, tag, round) 
    );
'''
util.exec_sql(sql_txt)

In [ ]:
# convenience table for player names
sql_txt = '''
    drop table if exists players cascade;
    create table players (
         player_id    integer PRIMARY KEY,
         name   varchar(40) NOT NULL CHECK (name <> '')
);
'''
exec_sql(sql_txt)

In [ ]:
sql_txt = '''
    insert into players (player_id, name) 
        select distinct player_id, player_first_name || ' ' ||  player_last_name from strokes;
'''
exec_sql(sql_txt)

In [ ]:
# convenience table for courses
sql_txt = '''
create table courses (
     year        integer,
     course_id    integer,
     name   varchar(40) NOT NULL CHECK (name <> ''),
     primary key (year, course_id) 
);
'''
exec_sql(sql_txt)

In [ ]:
sql_txt = '''
    insert into courses
        select distinct year, course_id, course_name from strokes
'''
exec_sql(sql_txt)

In [ ]:
# convenience table for tournaments
sql_txt = '''
    create table tournaments (
        year        integer,
        permanent_tournament_id    integer,
        name   varchar(100) NOT NULL CHECK (name <> ''),
        primary key (year, permanent_tournament_id) 
    );
'''
exec_sql(sql_txt)

In [ ]:
sql_txt = '''
    insert into tournaments
        select distinct year, permanent_tournament_id, tournament_name from strokes;
'''
exec_sql(sql_txt)